In [20]:
from torchsummary import summary
from torch import nn, einsum
import torch
from einops import rearrange, reduce, repeat

class Attention(nn.Module):
    def __init__(
        self, 
        dim, 
        heads = 4, 
        dim_head = 1
    ):
        super().__init__()
        self.scale = dim_head ** -0.5
        self.heads = heads
        hidden_dim = dim_head * heads

        self.to_qkv = nn.Conv2d(dim, hidden_dim * 3, 1, bias = False)
        self.to_out = nn.Conv2d(hidden_dim, dim, 1)
        self.net = nn.LazyLinear(3)

    def forward(self, x):
        b, c, h, w = x.shape
        nn.Linear(2,3)(torch.tensor([1,2.]))
        self.net(torch.tensor([1.]))
        qkv = self.to_qkv(x).chunk(3, dim = 1)
        print(qkv[0].shape)
        q, k, v = map(lambda t: rearrange(t, 'b (h c) x y -> b h c (x y)', h = self.heads), qkv)

        q = q * self.scale

        sim = einsum('b h d i, b h d j -> b h i j', q, k)
        attn = sim.softmax(dim = -1)
        out = einsum('b h i j, b h d j -> b h i d', attn, v)

        out = rearrange(out, 'b h (x y) d -> b (h d) x y', x = h, y = w)
        return self.to_out(out)

A = Attention(5)
summary(A, input_size=(5, 64, 64), batch_size=7)

torch.Size([2, 4, 64, 64])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                        [7]               6
            Conv2d-2            [7, 12, 64, 64]              60
            Conv2d-3             [7, 5, 64, 64]              25
Total params: 91
Trainable params: 91
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.55
Forward/backward pass size (MB): 3.72
Params size (MB): 0.00
Estimated Total Size (MB): 4.27
----------------------------------------------------------------
